# Import Necessary Packages

In [1]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [2]:
OPENAI_API_KEY = ""
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_1312\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [11]:
host = 'localhost'
port = '3307'
username = 'root'
#password = ''
database_schema = 'bank_customers'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['p6_uk_bank_customers'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [12]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [17]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)

    system_message = """You are a bank database expert specializing in bank customers.
    Your task is to answer user's questions by providing relevant information from a database of p6_uk_bank_customers.
    The database contains the following information for each customer
    - ID of the Customer
    - Name of the Customer
    - Surname of the Customer
    - Gender of the Customer
    - Age of the Customer
    - Region of the Customer Where he/she lives 
    - Job Classification 
    - Date of joined
    - Balance amount of the Customer

    Example:

    Input:
    Which customer has low balance amount?

    Context:
    The customer who has the low balance amount is:
    1. Customer A - Balance amount: 10000
    2. Customer B - Balance amount: 1000

    Output:
    The low balance amount contains Customer A (Balance amount: 10000) and Customer B (Balance amount: 1000).
    """

    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
    
        Context:
        {db_context}
    
        Output"
        """
    )
    message = [
        SystemMessage(content=system_message),
        human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(message).content
    return response
    
       

In [29]:
generate("Provide names of all people from England")



> Entering new SQLDatabaseChain chain...
Provide names of all people from England
SQLQuery:SELECT `Name`, `Surname` 
FROM p6_uk_bank_customers 
WHERE `Region` = 'England'
LIMIT 5;
SQLResult: [('Simon', 'Walsh'), ('Liam', 'Brown'), ('Deirdre', 'Pullman'), ('Dorothy', 'Thomson'), ('Dominic', 'Parr')]
Answer:Simon Walsh, Liam Brown, Deirdre Pullman, Dorothy Thomson, Dominic Parr
> Finished chain.


'The names of all people from England are:\n1. Simon Walsh\n2. Liam Brown\n3. Deirdre Pullman\n4. Dorothy Thomson\n5. Dominic Parr'

In [28]:
generate("Provide count of all Male people")



> Entering new SQLDatabaseChain chain...
Provide count of all Male people
SQLQuery:SELECT COUNT(`Customer ID`) AS Male_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Male';
SQLResult: [(2165,)]
Answer:There are 2165 Male people in the database.
> Finished chain.


'The count of all Male people in the database is 2165.'

In [27]:
generate("Provide count of all female who are from Wales")



> Entering new SQLDatabaseChain chain...
Provide count of all female who are from Wales
SQLQuery:SELECT COUNT(`Customer ID`) AS FemaleCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND Region = 'Wales';
SQLResult: [(282,)]
Answer:There are 282 females from Wales.
> Finished chain.


'The count of all females who are from Wales is 282.'

In [26]:
generate("Provide count of people with White color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with White color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `White Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(1955,)]
Answer:There are 1955 people with White Collar jobs.
> Finished chain.


'The count of people with White Collar jobs is 1955.'

In [25]:
generate("Provide count of people with Blue Color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with Blue Color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Blue Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'Blue Collar';
SQLResult: [(1049,)]
Answer:Blue Collar Count: 1
> Finished chain.


'The count of people with Blue Collar jobs is 1.'

In [24]:
generate("Provide count of people whose balance is above 5000")



> Entering new SQLDatabaseChain chain...
Provide count of people whose balance is above 5000
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`
FROM p6_uk_bank_customers
WHERE `Balance` > 5000;
SQLResult: [(3690,)]
Answer:There are 3690 people whose balance is above 5000.
> Finished chain.


'The count of people whose balance is above 5000 is 3690.'

In [23]:
generate("Provide names of all people who are white color and age above 35")



> Entering new SQLDatabaseChain chain...
Provide names of all people who are white color and age above 35
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar' AND `Age` > 35
LIMIT 5;
SQLResult: [('Liam',), ('Dominic',), ('Dominic',), ('Benjamin',), ('Nicholas',)]
Answer:Liam, Dominic, Benjamin, and Nicholas are the names of people who are white collar and above the age of 35.
> Finished chain.


'The names of people who are white collar and above the age of 35 are Liam, Dominic, Benjamin, and Nicholas.'

In [22]:
generate("Provide count of all female people who are with White color job")



> Entering new SQLDatabaseChain chain...
Provide count of all female people who are with White color job
SQLQuery:SELECT COUNT(`Customer ID`) AS Female_White_Collar_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND `Job Classification` = 'White Collar';
SQLResult: [(1010,)]
Answer:There are 1010 female people with a White Collar job.
> Finished chain.


'The count of all female people with a White Collar job is 1010.'

In [21]:
generate("Provide count of people from Wales and doing white color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people from Wales and doing white color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`
FROM p6_uk_bank_customers
WHERE Region = 'Wales' AND `Job Classification` = 'White Collar';
SQLResult: [(305,)]
Answer:There are 305 people from Wales doing white collar jobs.
> Finished chain.


'The count of people from Wales doing white collar jobs is 305.'

In [20]:
generate("Provide average age of people with white color job")



> Entering new SQLDatabaseChain chain...
Provide average age of people with white color job
SQLQuery:SELECT AVG(`Age`) AS `Average Age`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(Decimal('35.2793'),)]
Answer:The average age of people with a white collar job is 35.2793.
> Finished chain.


'The average age of people with a white collar job is 35.2793.'

In [19]:
generate("provide maximum balance and name of the person")



> Entering new SQLDatabaseChain chain...
provide maximum balance and name of the person
SQLQuery:SELECT `Name`, `Balance` 
FROM p6_uk_bank_customers 
ORDER BY `Balance` DESC 
LIMIT 1;
SQLResult: [('Dorothy', Decimal('183467.70'))]
Answer:The person with the maximum balance is Dorothy with a balance of 183467.70.
> Finished chain.


'The person with the maximum balance is Dorothy with a balance of 183467.70.'

In [18]:
generate("Provide minimum balance and name, job classification")



> Entering new SQLDatabaseChain chain...
Provide minimum balance and name, job classification
SQLQuery:SELECT `Name`, `Job Classification`, MIN(`Balance`) AS `Minimum Balance`
FROM p6_uk_bank_customers
GROUP BY `Name`, `Job Classification`
ORDER BY `Minimum Balance`
LIMIT 5;
SQLResult: [('Jane', 'White Collar', Decimal('11.52')), ('Olivia', 'White Collar', Decimal('21.03')), ('Warren', 'Blue Collar', Decimal('69.01')), ('Megan', 'Blue Collar', Decimal('69.78')), ('Stewart', 'Blue Collar', Decimal('77.46'))]
Answer:The customers with the minimum balance along with their names and job classifications are:
1. Jane - White Collar (Minimum Balance: 11.52)
2. Olivia - White Collar (Minimum Balance: 21.03)
3. Warren - Blue Collar (Minimum Balance: 69.01)
4. Megan - Blue Collar (Minimum Balance: 69.78)
5. Stewart - Blue Collar (Minimum Balance: 77.46)
> Finished chain.


C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_1312\3910421833.py:45: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(message).content


'The customers with the minimum balance along with their names and job classifications are:\n1. Jane - White Collar (Minimum Balance: 11.52)\n2. Olivia - White Collar (Minimum Balance: 21.03)\n3. Warren - Blue Collar (Minimum Balance: 69.01)\n4. Megan - Blue Collar (Minimum Balance: 69.78)\n5. Stewart - Blue Collar (Minimum Balance: 77.46)'